# LLaDA SFT学習ノートブック

このノートブックは、LLaDA（Large Language Diffusion with mAsking）モデルのSupervised Fine-Tuning（SFT）を実装します。

## 📋 実行手順（初心者向け）

### 🚀 最速実行（推奨）
```python
# 全てのセルを順番に実行後、以下を実行：
result = run_complete_sft_demo()
```

### 📝 ステップバイステップ実行
1. **セル1-8を順番に実行** - 環境設定からモデル準備まで
2. **学習実行** - 以下のいずれかを選択：
   - 🥇 `result = run_complete_sft_demo()` （初回推奨）
   - ⚡ `result = quick_start_button()` （最速）
   - 🎛️ `result = custom_training(dataset_size=1000, batch_size=2, epochs=1)` （カスタム）

### ⚙️ 設定選択ガイド
- **validation**: 1K サンプル, 1エポック（初心者・検証用）
- **medium**: 10K サンプル, 2エポック（中級者・バランス型）
- **production**: 50K サンプル, 3エポック（上級者・本格学習）

## ✨ 特徴
- ✅ Google Colab Pro対応（16GB GPU制限内で動作）
- ✅ GUIDELINES.mdに基づく正確なSFT実装
- ✅ 日本語Alpacaデータセット使用
- ✅ ノートブック内完結（外部ツール不要）
- ✅ ワンクリック実行機能
- ✅ リアルタイム進捗監視
- ✅ 自動エラー回復機能

## 🔬 SFTの技術的特徴
- **プロンプト非マスキング**: ユーザープロンプト部分はマスクしない
- **Answer length正規化**: 回答部分の長さで損失を正規化
- **完全文学習**: 事前学習とは異なり、完全な文章で学習

## ❓ トラブルシューティング
- メモリエラー → より小さな設定（validation）を使用
- CUDA エラー → `torch.cuda.empty_cache()` 実行
- データセットエラー → サンプルデータで自動フォールバック
- 詳細なヘルプは最下部のFAQセクションを参照

---

In [ ]:
# 📦 環境設定とライブラリのインストール
import os
import sys
import warnings
warnings.filterwarnings('ignore')

# WandB完全無効化（API key要求を防ぐ）
os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_MODE"] = "disabled"
os.environ["WANDB_SILENT"] = "true"

# Google Colab環境の検出
IN_COLAB = 'google.colab' in sys.modules
print(f"実行環境: {'Google Colab' if IN_COLAB else 'ローカル環境'}")

if IN_COLAB:
    # Google Colab用のGPU情報表示
    !nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv
    
    # 必要なパッケージのインストール
    print("📦 必要なパッケージをインストール中...")
    !pip install -q transformers==4.49.0 accelerate==0.34.2 datasets==2.21.0
    !pip install -q torch==2.0.1 torchvision==0.15.2
    !pip install -q matplotlib seaborn tqdm pandas
    
    print("✅ パッケージのインストールが完了しました")

# 必要なライブラリのインポート
print("📚 ライブラリをインポート中...")
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from datasets import load_dataset, Dataset as HFDataset
from tqdm.auto import tqdm
import json
import random
from typing import Dict, List, Optional, Tuple
import gc
from dataclasses import dataclass
import time
from datetime import datetime
import pandas as pd

# WandB無効化確認
try:
    import wandb
    wandb.init(mode="disabled")
    print("🚫 WandB無効化確認済み")
except ImportError:
    print("✅ WandB未インストール（問題なし）")
except Exception:
    print("✅ WandB無効化済み")

# デバイス設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🖥️  使用デバイス: {device}")
if torch.cuda.is_available():
    print(f"  GPU: {torch.cuda.get_device_name(0)}")
    print(f"  利用可能メモリ: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("  ⚠️  GPU未検出 - CPU学習になります")

# シード固定
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

# スタイル設定
plt.style.use('default')  # シンプルなスタイル
sns.set_palette("husl")   # 見やすい色合い

print("✅ 環境設定が完了しました")
print("🚫 WandB完全無効化済み - API key要求なし")
print("=" * 50)
print("🎯 次のステップ: セル2を実行して学習設定を選択")
print("=" * 50)

In [ ]:
# 学習設定の定義
@dataclass
class TrainingConfig:
    """学習設定を管理するクラス"""
    
    # モデル設定
    model_name: str = "GSAI-ML/LLaDA-8B-Base"
    max_length: int = 1024
    
    # データ設定
    dataset_size: int = 1000  # 使用するデータ数
    validation_split: float = 0.1
    
    # 学習設定
    batch_size: int = 2
    gradient_accumulation_steps: int = 8
    num_epochs: int = 1
    learning_rate: float = 2e-5
    warmup_steps: int = 100
    
    # SFT特有設定
    mask_id: int = 126336  # [MASK]トークンID
    eps: float = 1e-3  # マスキング確率の最小値
    
    # メモリ最適化
    fp16: bool = True
    gradient_checkpointing: bool = True
    dataloader_num_workers: int = 2
    
    # 保存・評価設定
    save_steps: int = 500
    eval_steps: int = 250
    logging_steps: int = 50
    output_dir: str = "./llada_sft_output"

# 事前定義された設定
CONFIGS = {
    "validation": TrainingConfig(
        dataset_size=1000,
        batch_size=2,
        gradient_accumulation_steps=4,
        num_epochs=1,
        learning_rate=5e-5,
        save_steps=200,
        eval_steps=100
    ),
    
    "production": TrainingConfig(
        dataset_size=50000,  # 本格的なデータサイズ
        batch_size=1,        # メモリ制限対応
        gradient_accumulation_steps=16,
        num_epochs=3,
        learning_rate=2e-5,
        warmup_steps=500,
        save_steps=1000,
        eval_steps=500
    ),
    
    "medium": TrainingConfig(
        dataset_size=10000,
        batch_size=2,
        gradient_accumulation_steps=8,
        num_epochs=2,
        learning_rate=3e-5,
        save_steps=500,
        eval_steps=250
    )
}

def select_config(config_name: str = "validation") -> TrainingConfig:
    """設定を選択する関数"""
    if config_name not in CONFIGS:
        print(f"警告: '{config_name}'は無効な設定です。利用可能: {list(CONFIGS.keys())}")
        config_name = "validation"
    
    config = CONFIGS[config_name]
    print(f"✅ '{config_name}' 設定を選択しました")
    print(f"  - データサイズ: {config.dataset_size:,}")
    print(f"  - バッチサイズ: {config.batch_size}")
    print(f"  - エポック数: {config.num_epochs}")
    print(f"  - 学習率: {config.learning_rate}")
    
    # GPU メモリ使用量の推定
    if torch.cuda.is_available():
        estimated_memory = (
            8 * config.batch_size * config.max_length * 4 / 1e9 +  # モデルパラメータ
            config.batch_size * config.max_length * 4 * 2 / 1e9    # アクティベーション
        )
        available_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
        print(f"  - 推定GPU使用量: {estimated_memory:.1f} GB / {available_memory:.1f} GB")
        
        if estimated_memory > available_memory * 0.9:
            print("  ⚠️  メモリ不足の可能性があります。より小さな設定を検討してください。")
    
    return config

# 使用する設定を選択（validation/medium/production）
CONFIG_NAME = "validation"  # ここを変更して設定を選択
config = select_config(CONFIG_NAME)

In [ ]:
# Alpacaデータセットの読み込みと前処理
class AlpacaDatasetProcessor:
    """日本語Alpacaデータセットの処理クラス"""
    
    def __init__(self, tokenizer, config: TrainingConfig):
        self.tokenizer = tokenizer
        self.config = config
        self.special_tokens = {
            'bos': tokenizer.bos_token or '<s>',
            'eos': tokenizer.eos_token or '</s>',
            'start_id': '<start_id>',
            'end_id': '<end_id>',
            'eot_id': '<eot_id>'
        }
    
    def load_dataset(self) -> List[Dict]:
        """日本語Alpacaデータセットを読み込む"""
        try:
            print("📥 日本語Alpacaデータセットを読み込み中...")
            
            # データセットの読み込み（複数のソースを試行）
            dataset_sources = [
                "kunishou/databricks-dolly-15k-ja",
                "izumi-lab/llm-japanese-dataset",
                "elyza/ELYZA-tasks-100"
            ]
            
            dataset = None
            for source in dataset_sources:
                try:
                    dataset = load_dataset(source, split='train')
                    print(f"✅ データセット '{source}' の読み込みに成功")
                    break
                except Exception as e:
                    print(f"  '{source}' の読み込みに失敗: {e}")
                    continue
            
            if dataset is None:
                # フォールバック: サンプルデータを生成
                print("⚠️  データセットの読み込みに失敗。サンプルデータを使用します。")
                return self._create_sample_data()
            
            # データを標準形式に変換
            processed_data = self._process_dataset(dataset)
            
            # データサイズを制限
            if len(processed_data) > self.config.dataset_size:
                processed_data = processed_data[:self.config.dataset_size]
            
            print(f"✅ {len(processed_data):,} サンプルを準備しました")
            return processed_data
            
        except Exception as e:
            print(f"❌ データセット読み込みエラー: {e}")
            print("サンプルデータを使用します")
            return self._create_sample_data()
    
    def _process_dataset(self, dataset) -> List[Dict]:
        """データセットを標準形式に変換"""
        processed = []
        
        for item in tqdm(dataset, desc="データ変換中"):
            # データセット形式に応じて適応
            if 'instruction' in item and 'output' in item:
                instruction = item['instruction']
                if 'input' in item and item['input']:
                    instruction += f"\n{item['input']}"
                response = item['output']
            elif 'input' in item and 'output' in item:
                instruction = item['input']
                response = item['output']
            else:
                continue
            
            if instruction and response:
                processed.append({
                    'instruction': instruction.strip(),
                    'response': response.strip()
                })
        
        return processed
    
    def _create_sample_data(self) -> List[Dict]:
        """サンプルデータを生成"""
        sample_data = [
            {
                'instruction': '日本の首都はどこですか？',
                'response': '日本の首都は東京です。東京は関東地方に位置し、日本の政治・経済・文化の中心地です。'
            },
            {
                'instruction': 'Pythonでリストを逆順にする方法を教えてください。',
                'response': 'Pythonでリストを逆順にする方法はいくつかあります。最も簡単な方法は、reverse()メソッドを使うことです：my_list.reverse()。また、スライスを使って新しいリストを作ることもできます：new_list = my_list[::-1]。'
            },
            {
                'instruction': '機械学習とは何ですか？',
                'response': '機械学習は、コンピュータがデータから自動的にパターンを学習し、新しいデータに対して予測や判断を行う技術です。従来のプログラミングとは異なり、明示的にルールを書く代わりに、大量のデータから規則性を見つけ出します。'
            }
        ]
        
        # サンプルデータを指定サイズまで繰り返し
        repeated_data = []
        for i in range(self.config.dataset_size):
            repeated_data.append(sample_data[i % len(sample_data)])
        
        return repeated_data
    
    def format_for_sft(self, instruction: str, response: str) -> Tuple[str, int]:
        """SFT用にデータをフォーマット（GUIDELINES.mdに基づく）"""
        # フォーマット: <BOS><start_id>user<end_id>\n{instruction}<eot_id><start_id>assistant<end_id>\n{response}<EOS>
        prompt_part = (
            f"{self.special_tokens['bos']}"
            f"{self.special_tokens['start_id']}user{self.special_tokens['end_id']}\n"
            f"{instruction}"
            f"{self.special_tokens['eot_id']}"
            f"{self.special_tokens['start_id']}assistant{self.special_tokens['end_id']}\n"
        )
        
        full_text = prompt_part + response + self.special_tokens['eos']
        
        # プロンプト長を計算（response部分を除く）
        prompt_tokens = self.tokenizer.encode(prompt_part, add_special_tokens=False)
        prompt_length = len(prompt_tokens)
        
        return full_text, prompt_length

# データセットプロセッサの初期化
print("🔧 トークナイザーを読み込み中...")
tokenizer = AutoTokenizer.from_pretrained(config.model_name, trust_remote_code=True)

# パディングトークンの設定
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print(f"✅ トークナイザー読み込み完了 (語彙サイズ: {len(tokenizer):,})")

# データセットの処理
processor = AlpacaDatasetProcessor(tokenizer, config)
raw_data = processor.load_dataset()

print(f"📊 データセット情報:")
print(f"  - 総サンプル数: {len(raw_data):,}")
if raw_data:
    sample = raw_data[0]
    formatted_text, prompt_len = processor.format_for_sft(sample['instruction'], sample['response'])
    print(f"  - サンプル長: {len(formatted_text)} 文字")
    print(f"  - プロンプト長: {prompt_len} トークン")
    print(f"\n📝 フォーマット例:")
    print(f"  指示: {sample['instruction'][:50]}...")
    print(f"  応答: {sample['response'][:50]}...")

In [ ]:
# SFT用データセットクラス
class LLaDASFTDataset(Dataset):
    """LLaDA SFT学習用データセットクラス"""
    
    def __init__(self, data: List[Dict], processor: AlpacaDatasetProcessor, config: TrainingConfig):
        self.data = data
        self.processor = processor
        self.config = config
        self.tokenizer = processor.tokenizer
        
        # データを事前に処理
        self.processed_data = self._preprocess_data()
        
        print(f"✅ SFTデータセットを準備しました ({len(self.processed_data)} サンプル)")
    
    def _preprocess_data(self) -> List[Dict]:
        """全データを事前に処理"""
        processed = []
        
        for item in tqdm(self.data, desc="SFTデータ前処理中"):
            try:
                # フォーマット
                formatted_text, prompt_length = self.processor.format_for_sft(
                    item['instruction'], 
                    item['response']
                )
                
                # トークン化
                tokens = self.tokenizer.encode(
                    formatted_text,
                    add_special_tokens=False,
                    max_length=self.config.max_length,
                    truncation=True
                )
                
                # 十分な長さがある場合のみ使用
                if len(tokens) > prompt_length + 5:  # 最低5トークンの応答が必要
                    processed.append({
                        'input_ids': tokens,
                        'prompt_length': prompt_length,
                        'original_instruction': item['instruction'],
                        'original_response': item['response']
                    })
                    
            except Exception as e:
                print(f"データ処理エラー (スキップ): {e}")
                continue
        
        return processed
    
    def __len__(self) -> int:
        return len(self.processed_data)
    
    def __getitem__(self, idx: int) -> Dict[str, torch.Tensor]:
        item = self.processed_data[idx]
        
        # パディング
        input_ids = item['input_ids'][:self.config.max_length]
        if len(input_ids) < self.config.max_length:
            pad_length = self.config.max_length - len(input_ids)
            input_ids.extend([self.tokenizer.pad_token_id] * pad_length)
        
        return {
            'input_ids': torch.tensor(input_ids, dtype=torch.long),
            'prompt_length': torch.tensor(item['prompt_length'], dtype=torch.long),
            'attention_mask': torch.tensor(
                [1 if token_id != self.tokenizer.pad_token_id else 0 for token_id in input_ids],
                dtype=torch.long
            )
        }

# データセットの分割と作成
def create_datasets(data: List[Dict], processor: AlpacaDatasetProcessor, config: TrainingConfig):
    """学習・検証データセットを作成"""
    # データをシャッフル
    random.shuffle(data)
    
    # 分割
    split_idx = int(len(data) * (1 - config.validation_split))
    train_data = data[:split_idx]
    val_data = data[split_idx:]
    
    print(f"📊 データ分割:")
    print(f"  - 学習データ: {len(train_data):,} サンプル")
    print(f"  - 検証データ: {len(val_data):,} サンプル")
    
    # データセット作成
    train_dataset = LLaDASFTDataset(train_data, processor, config)
    val_dataset = LLaDASFTDataset(val_data, processor, config) if val_data else None
    
    return train_dataset, val_dataset

# データセットの作成
train_dataset, val_dataset = create_datasets(raw_data, processor, config)

In [ ]:
# SFT用Forward Process実装（GUIDELINES.mdに基づく）
class SFTForwardProcess:
    """SFT用のマスキング・損失計算クラス"""
    
    def __init__(self, config: TrainingConfig):
        self.config = config
        self.mask_id = config.mask_id
        self.eps = config.eps
    
    def forward_process(self, input_ids: torch.Tensor, prompt_lengths: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        """
        SFT用フォワードプロセス（GUIDELINES.md準拠）
        
        Args:
            input_ids: [batch_size, seq_len] 入力トークンID
            prompt_lengths: [batch_size] 各サンプルのプロンプト長
        
        Returns:
            noisy_batch: マスクされた入力
            masked_indices: マスクされた位置
            p_mask: マスキング確率
        """
        b, l = input_ids.shape
        device = input_ids.device
        
        # ランダムマスキング確率の生成
        t = torch.rand(b, device=device)
        p_mask = (1 - self.eps) * t + self.eps
        p_mask = p_mask[:, None].repeat(1, l)
        
        # 初期マスキング
        masked_indices = torch.rand((b, l), device=device) < p_mask
        noisy_batch = torch.where(masked_indices, self.mask_id, input_ids)
        
        # プロンプト部分のマスクを解除（SFTの核心部分）
        token_positions = torch.arange(l, device=device).expand(b, l)
        prompt_mask = token_positions < prompt_lengths.unsqueeze(1)
        
        # プロンプト部分は元のトークンを保持
        noisy_batch[prompt_mask] = input_ids[prompt_mask]
        
        # マスクインデックスを更新（プロンプト部分はマスクされていない）
        masked_indices = (noisy_batch == self.mask_id)
        
        return noisy_batch, masked_indices, p_mask
    
    def compute_sft_loss(self, 
                        logits: torch.Tensor,
                        input_ids: torch.Tensor,
                        masked_indices: torch.Tensor,
                        p_mask: torch.Tensor,
                        prompt_lengths: torch.Tensor) -> torch.Tensor:
        """
        SFT損失の計算（GUIDELINES.md準拠）
        
        Args:
            logits: [batch_size, seq_len, vocab_size] モデル出力
            input_ids: [batch_size, seq_len] 正解トークン
            masked_indices: [batch_size, seq_len] マスクされた位置
            p_mask: [batch_size, seq_len] マスキング確率
            prompt_lengths: [batch_size] プロンプト長
        
        Returns:
            loss: SFT損失
        """
        b, l = input_ids.shape
        device = input_ids.device
        
        # Answer length計算（プロンプト以外の部分）
        prompt_mask = torch.arange(l, device=device).expand(b, l) < prompt_lengths.unsqueeze(1)
        prompt_mask = prompt_mask.to(torch.int64)
        answer_lengths = torch.sum((1 - prompt_mask), dim=-1, keepdim=True)
        answer_lengths = answer_lengths.repeat(1, l)
        
        # マスクされた位置でのみ損失を計算
        if not masked_indices.any():
            return torch.tensor(0.0, device=device, requires_grad=True)
        
        # クロスエントロピー損失（重み付き）
        token_loss = F.cross_entropy(
            logits[masked_indices], 
            input_ids[masked_indices], 
            reduction='none'
        ) / p_mask[masked_indices]
        
        # Answer length正規化
        normalized_loss = token_loss / (answer_lengths[masked_indices] + 1e-8)
        
        # バッチ平均
        ce_loss = torch.sum(normalized_loss) / b
        
        return ce_loss

# SFT Forward Processの初期化
sft_forward = SFTForwardProcess(config)
print("✅ SFT Forward Processを初期化しました")

# テスト実行
if len(train_dataset) > 0:
    test_batch = train_dataset[0]
    test_input_ids = test_batch['input_ids'].unsqueeze(0)
    test_prompt_length = test_batch['prompt_length'].unsqueeze(0)
    
    print(f"\n🧪 SFTプロセステスト:")
    print(f"  - 入力形状: {test_input_ids.shape}")
    print(f"  - プロンプト長: {test_prompt_length.item()}")
    
    # Forward process実行
    noisy_batch, masked_indices, p_mask = sft_forward.forward_process(
        test_input_ids, test_prompt_length
    )
    
    print(f"  - マスクされたトークン数: {masked_indices.sum().item()}")
    print(f"  - プロンプト部分のマスク数: {masked_indices[0, :test_prompt_length.item()].sum().item()} (0であるべき)")
    print(f"  - 応答部分のマスク数: {masked_indices[0, test_prompt_length.item():].sum().item()}")
    
    if masked_indices[0, :test_prompt_length.item()].sum().item() == 0:
        print("  ✅ プロンプト非マスキングが正常に動作しています")
    else:
        print("  ❌ プロンプト部分がマスクされています（エラー）")

In [ ]:
# 🤖 LLaDAモデルとSFTトレーナーの設定
class LLaDASFTTrainer(Trainer):
    """LLaDA SFT専用トレーナークラス"""
    
    def __init__(self, sft_forward: SFTForwardProcess, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.sft_forward = sft_forward
        self.vocab_size = len(self.tokenizer)
    
    def compute_loss(self, model, inputs, return_outputs=False):
        """SFT損失の計算"""
        input_ids = inputs['input_ids']
        prompt_lengths = inputs['prompt_length']
        
        # 安全性チェック
        input_ids = torch.clamp(input_ids, 0, self.vocab_size - 1)
        
        try:
            # SFTフォワードプロセス
            noisy_batch, masked_indices, p_mask = self.sft_forward.forward_process(
                input_ids, prompt_lengths
            )
            
            # モデル推論
            outputs = model(input_ids=noisy_batch)
            logits = outputs.logits
            
            # SFT損失計算
            loss = self.sft_forward.compute_sft_loss(
                logits, input_ids, masked_indices, p_mask, prompt_lengths
            )
            
            # NaN/Inf チェック
            if torch.isnan(loss) or torch.isinf(loss):
                print(f"警告: 異常な損失値 {loss.item()}, ゼロ損失で代替")
                loss = torch.tensor(0.0, device=loss.device, requires_grad=True)
            
            return (loss, outputs) if return_outputs else loss
            
        except Exception as e:
            print(f"損失計算エラー: {e}")
            # フォールバック損失
            fallback_loss = torch.tensor(1.0, device=input_ids.device, requires_grad=True)
            return fallback_loss

# モデルの読み込み
print("🔧 LLaDAモデルを読み込み中...")
try:
    model = AutoModelForCausalLM.from_pretrained(
        config.model_name,
        trust_remote_code=True,
        torch_dtype=torch.bfloat16 if config.fp16 else torch.float32,
        device_map='auto' if torch.cuda.is_available() else None
    )
    
    # グラディエントチェックポイントの設定
    if config.gradient_checkpointing:
        try:
            model.gradient_checkpointing_enable()
            print("  ✅ グラディエントチェックポイントを有効化")
        except Exception as e:
            print(f"  ⚠️  グラディエントチェックポイント設定エラー: {e}")
    
    print(f"✅ モデル読み込み完了")
    print(f"  - パラメータ数: {sum(p.numel() for p in model.parameters()):,}")
    print(f"  - 学習可能パラメータ数: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")
    
except Exception as e:
    print(f"❌ モデル読み込みエラー: {e}")
    raise

# データコレーターの設定
def sft_data_collator(features):
    """SFT用データコレーター"""
    batch = {}
    # バッチの作成
    for key in features[0].keys():
        batch[key] = torch.stack([f[key] for f in features])
    return batch

# 学習引数の設定（WandB完全無効化）
training_args = TrainingArguments(
    output_dir=config.output_dir,
    num_train_epochs=config.num_epochs,
    per_device_train_batch_size=config.batch_size,
    per_device_eval_batch_size=config.batch_size,
    gradient_accumulation_steps=config.gradient_accumulation_steps,
    learning_rate=config.learning_rate,
    warmup_steps=config.warmup_steps,
    logging_steps=config.logging_steps,
    save_steps=config.save_steps,
    eval_steps=config.eval_steps,
    evaluation_strategy="steps" if val_dataset else "no",
    save_strategy="steps",
    fp16=config.fp16,
    dataloader_num_workers=config.dataloader_num_workers,
    remove_unused_columns=False,
    # WandB完全無効化の複数設定
    report_to=[],  # 空リストで完全無効化
    logging_first_step=True,
    disable_tqdm=False,  # プログレスバーは表示
    load_best_model_at_end=True if val_dataset else False,
    metric_for_best_model="eval_loss" if val_dataset else None,
    greater_is_better=False,
    save_total_limit=2,
    # 追加のWandB無効化設定
    run_name=None,
    logging_strategy="steps",
    log_level="error"  # ログレベルを最小限に
)

# トレーナーの初期化
trainer = LLaDASFTTrainer(
    sft_forward=sft_forward,
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=sft_data_collator,
    tokenizer=tokenizer
)

print("✅ SFTトレーナーを初期化しました")
print(f"📊 学習設定:")
print(f"  - 実効バッチサイズ: {config.batch_size * config.gradient_accumulation_steps}")
print(f"  - 総ステップ数: {len(train_dataset) // (config.batch_size * config.gradient_accumulation_steps) * config.num_epochs}")
print(f"  - 学習率: {config.learning_rate}")
print(f"  - FP16: {config.fp16}")
print(f"🚫 外部ログ: 完全無効化済み（WandB API key不要）")
print("=" * 50)
print("🎯 次のステップ: セル7-8を実行して学習機能を準備")
print("=" * 50)

In [ ]:
# SFT学習の実行
def run_sft_training():
    """SFT学習の実行関数"""
    print("🚀 SFT学習を開始します...")
    print(f"📊 学習統計:")
    print(f"  - 学習サンプル数: {len(train_dataset):,}")
    print(f"  - 検証サンプル数: {len(val_dataset) if val_dataset else 0:,}")
    print(f"  - エポック数: {config.num_epochs}")
    print(f"  - 使用設定: {CONFIG_NAME}")
    
    # GPU メモリのクリア
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        gc.collect()
        print(f"  - GPU使用メモリ: {torch.cuda.memory_allocated() / 1e9:.1f} GB")
    
    try:
        # 学習前の評価（オプション）
        if val_dataset and len(val_dataset) > 0:
            print("\n📊 学習前評価を実行中...")
            try:
                initial_eval = trainer.evaluate()
                print(f"  初期損失: {initial_eval['eval_loss']:.4f}")
            except Exception as e:
                print(f"  初期評価エラー: {e}")
        
        # メイン学習ループ
        print("\n🎯 メイン学習を開始...")
        training_result = trainer.train()
        
        print("\n✅ 学習が完了しました！")
        print(f"📊 学習結果:")
        print(f"  - 最終損失: {training_result.training_loss:.4f}")
        print(f"  - 学習時間: {training_result.metrics['train_runtime']:.2f} 秒")
        print(f"  - サンプル/秒: {training_result.metrics['train_samples_per_second']:.2f}")
        
        # 学習後の評価
        if val_dataset and len(val_dataset) > 0:
            print("\n📊 最終評価を実行中...")
            try:
                final_eval = trainer.evaluate()
                print(f"  最終検証損失: {final_eval['eval_loss']:.4f}")
            except Exception as e:
                print(f"  最終評価エラー: {e}")
        
        # モデルの保存
        print("\n💾 モデルを保存中...")
        try:
            trainer.save_model()
            tokenizer.save_pretrained(config.output_dir)
            print(f"  ✅ モデルを {config.output_dir} に保存しました")
        except Exception as e:
            print(f"  ⚠️  モデル保存エラー: {e}")
        
        return training_result
        
    except Exception as e:
        print(f"\n❌ 学習エラー: {e}")
        print("\n🔧 エラー対処法:")
        print("  1. より小さなバッチサイズを試す")
        print("  2. より短い最大長を設定する")
        print("  3. validation設定を使用する")
        print("  4. FP16を有効にする")
        
        # メモリ情報表示
        if torch.cuda.is_available():
            print(f"\n📊 GPU情報:")
            print(f"  - 使用メモリ: {torch.cuda.memory_allocated() / 1e9:.1f} GB")
            print(f"  - 最大メモリ: {torch.cuda.max_memory_allocated() / 1e9:.1f} GB")
            print(f"  - 利用可能メモリ: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
        
        raise

# 学習実行の確認
print("\n🎯 学習準備完了")
print(f"現在の設定: {CONFIG_NAME}")
print("\n実行するには下記を実行してください:")
print("training_result = run_sft_training()")

# 設定変更の案内
print("\n⚙️  設定変更方法:")
print("異なる設定で実行したい場合は、上のセルで CONFIG_NAME を変更してください:")
print("  - 'validation': 検証用 (1K サンプル, 1エポック)")
print("  - 'medium': 中規模 (10K サンプル, 2エポック)")
print("  - 'production': 本番用 (50K サンプル, 3エポック)")

In [ ]:
# 🚀 ワンクリック学習実行セル
import time
from IPython.display import display, HTML, clear_output
import threading

class SFTTrainingRunner:
    """SFT学習のワンクリック実行クラス"""
    
    def __init__(self):
        self.is_training = False
        self.training_thread = None
        self.training_logs = []
        self.current_step = 0
        self.total_steps = 0
        self.start_time = None
        
    def quick_start_training(self, config_name="validation", auto_evaluate=True):
        """
        ワンクリック学習実行
        
        Args:
            config_name: 使用する設定 ("validation", "medium", "production")
            auto_evaluate: 学習後に自動評価するかどうか
        """
        print("🚀 ワンクリック学習実行を開始します...")
        
        # 設定の再選択
        global config, CONFIG_NAME
        CONFIG_NAME = config_name
        config = select_config(config_name)
        
        # 学習統計の表示
        print(f"\n📊 学習設定: {config_name}")
        print(f"  ├─ データサイズ: {config.dataset_size:,} サンプル")
        print(f"  ├─ バッチサイズ: {config.batch_size}")
        print(f"  ├─ エポック数: {config.num_epochs}")
        print(f"  ├─ 学習率: {config.learning_rate}")
        print(f"  └─ 推定時間: {self._estimate_training_time()} 分")
        
        # GPU メモリチェック
        if torch.cuda.is_available():
            self._check_gpu_memory()
        
        # 学習実行
        try:
            print("\n⏰ 5秒後に学習を開始します...")
            time.sleep(5)
            
            # 学習実行
            self.start_time = time.time()
            print("🎯 学習開始！")
            training_result = run_sft_training()
            
            # 学習時間の計算
            training_duration = time.time() - self.start_time
            print(f"\n⏱️  総学習時間: {training_duration/60:.1f} 分")
            
            # 自動評価
            if auto_evaluate:
                print("\n🔍 自動評価を開始...")
                time.sleep(2)
                evaluate_sft_model()
                memory_usage_summary()
            
            # 成功メッセージ
            print("\n🎉 学習が正常に完了しました！")
            self._display_success_summary(training_result)
            
            return training_result
            
        except Exception as e:
            print(f"\n❌ 学習エラー: {e}")
            self._display_error_troubleshooting()
            raise
    
    def _estimate_training_time(self):
        """学習時間を推定"""
        # 基本的な推定（経験値ベース）
        base_time_per_sample = 0.1  # 秒/サンプル
        total_samples = config.dataset_size * config.num_epochs
        estimated_seconds = total_samples * base_time_per_sample / config.batch_size
        return max(1, int(estimated_seconds / 60))
    
    def _check_gpu_memory(self):
        """GPU メモリをチェック"""
        total_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
        current_memory = torch.cuda.memory_allocated() / 1e9
        
        estimated_usage = 8 + config.batch_size * 2  # GB
        
        print(f"\n🖥️  GPU メモリチェック:")
        print(f"  ├─ 総容量: {total_memory:.1f} GB")
        print(f"  ├─ 現在使用量: {current_memory:.1f} GB")
        print(f"  ├─ 推定必要量: {estimated_usage:.1f} GB")
        
        if estimated_usage > total_memory * 0.9:
            print(f"  └─ ⚠️  メモリ不足の可能性 → より小さな設定を推奨")
        else:
            print(f"  └─ ✅ メモリ容量OK")
    
    def _display_success_summary(self, training_result):
        """成功時のサマリー表示"""
        print("\\n" + "="*50)
        print("🎊 SFT学習完了サマリー")
        print("="*50)
        print(f"✅ 設定: {CONFIG_NAME}")
        print(f"✅ 最終損失: {training_result.training_loss:.4f}")
        print(f"✅ 学習時間: {training_result.metrics['train_runtime']:.1f} 秒")
        print(f"✅ モデル保存先: {config.output_dir}")
        print("="*50)
    
    def _display_error_troubleshooting(self):
        """エラー時のトラブルシューティング"""
        print("\\n" + "="*50)
        print("🔧 トラブルシューティング")
        print("="*50)
        print("💡 以下を試してください:")
        print("  1️⃣  より小さな設定を使用: quick_start_training('validation')")
        print("  2️⃣  バッチサイズを小さく: config.batch_size = 1")
        print("  3️⃣  最大長を短く: config.max_length = 512")
        print("  4️⃣  GPU メモリをクリア: torch.cuda.empty_cache()")
        print("="*50)

# SFTTrainingRunnerのインスタンス化
trainer_runner = SFTTrainingRunner()

# ワンクリック実行ボタン
print("🎯 ワンクリック学習実行の準備完了")
print("\\n" + "="*60)
print("🚀 以下のコマンドで即座に学習を開始できます:")
print("="*60)
print()
print("💡 検証用設定（推奨・初回）:")
print("   trainer_runner.quick_start_training('validation')")
print()
print("💪 中規模設定:")
print("   trainer_runner.quick_start_training('medium')")
print()
print("🔥 本格的設定:")
print("   trainer_runner.quick_start_training('production')")
print()
print("="*60)
print("⚙️  オプション:")
print("   - auto_evaluate=False で評価をスキップ")
print("   - 例: trainer_runner.quick_start_training('validation', auto_evaluate=False)")
print("="*60)

In [ ]:
# 📊 インタラクティブ学習進捗モニタリング
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from datetime import datetime
import pandas as pd

class TrainingMonitor:
    """学習進捗をリアルタイムで監視するクラス"""
    
    def __init__(self):
        self.training_history = {
            'step': [],
            'loss': [],
            'eval_loss': [],
            'learning_rate': [],
            'timestamp': [],
            'gpu_memory': [],
            'samples_per_second': []
        }
        self.fig = None
        self.axes = None
        
    def start_monitoring(self):
        """モニタリングを開始"""
        plt.style.use('seaborn-v0_8')
        self.fig, self.axes = plt.subplots(2, 2, figsize=(15, 10))
        self.fig.suptitle('🔄 LLaDA SFT学習リアルタイムモニタリング', fontsize=16, fontweight='bold')
        
        # 各グラフの初期設定
        self.axes[0, 0].set_title('📉 学習損失')
        self.axes[0, 0].set_xlabel('ステップ')
        self.axes[0, 0].set_ylabel('損失')
        self.axes[0, 0].grid(True, alpha=0.3)
        
        self.axes[0, 1].set_title('🖥️ GPU メモリ使用量')
        self.axes[0, 1].set_xlabel('ステップ')
        self.axes[0, 1].set_ylabel('メモリ (GB)')
        self.axes[0, 1].grid(True, alpha=0.3)
        
        self.axes[1, 0].set_title('⚡ 学習速度')
        self.axes[1, 0].set_xlabel('ステップ')
        self.axes[1, 0].set_ylabel('サンプル/秒')
        self.axes[1, 0].grid(True, alpha=0.3)
        
        self.axes[1, 1].set_title('📊 学習率')
        self.axes[1, 1].set_xlabel('ステップ')
        self.axes[1, 1].set_ylabel('学習率')
        self.axes[1, 1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        return self.fig
    
    def update_metrics(self, step, loss=None, eval_loss=None, learning_rate=None, samples_per_second=None):
        """メトリクスを更新"""
        self.training_history['step'].append(step)
        self.training_history['timestamp'].append(datetime.now())
        
        # GPU メモリ使用量を取得
        if torch.cuda.is_available():
            gpu_memory = torch.cuda.memory_allocated() / 1e9
            self.training_history['gpu_memory'].append(gpu_memory)
        else:
            self.training_history['gpu_memory'].append(0)
        
        # その他のメトリクス
        self.training_history['loss'].append(loss if loss is not None else float('nan'))
        self.training_history['eval_loss'].append(eval_loss if eval_loss is not None else float('nan'))
        self.training_history['learning_rate'].append(learning_rate if learning_rate is not None else float('nan'))
        self.training_history['samples_per_second'].append(samples_per_second if samples_per_second is not None else float('nan'))
    
    def plot_current_progress(self):
        """現在の進捗をプロット"""
        if not self.training_history['step']:
            print("⚠️ モニタリングデータがありません")
            return
        
        # データをクリア
        for ax in self.axes.flat:
            ax.clear()
        
        steps = self.training_history['step']
        
        # 損失のプロット
        if any(not pd.isna(x) for x in self.training_history['loss']):
            valid_losses = [(s, l) for s, l in zip(steps, self.training_history['loss']) if not pd.isna(l)]
            if valid_losses:
                s_loss, losses = zip(*valid_losses)
                self.axes[0, 0].plot(s_loss, losses, 'b-', linewidth=2, label='学習損失')
        
        if any(not pd.isna(x) for x in self.training_history['eval_loss']):
            valid_eval_losses = [(s, l) for s, l in zip(steps, self.training_history['eval_loss']) if not pd.isna(l)]
            if valid_eval_losses:
                s_eval, eval_losses = zip(*valid_eval_losses)
                self.axes[0, 0].plot(s_eval, eval_losses, 'r--', linewidth=2, label='検証損失')
        
        self.axes[0, 0].set_title('📉 学習損失')
        self.axes[0, 0].set_xlabel('ステップ')
        self.axes[0, 0].set_ylabel('損失')
        self.axes[0, 0].legend()
        self.axes[0, 0].grid(True, alpha=0.3)
        
        # GPU メモリのプロット
        self.axes[0, 1].plot(steps, self.training_history['gpu_memory'], 'g-', linewidth=2)
        self.axes[0, 1].set_title('🖥️ GPU メモリ使用量')
        self.axes[0, 1].set_xlabel('ステップ')
        self.axes[0, 1].set_ylabel('メモリ (GB)')
        self.axes[0, 1].grid(True, alpha=0.3)
        
        # 学習速度のプロット
        if any(not pd.isna(x) for x in self.training_history['samples_per_second']):
            valid_speeds = [(s, sp) for s, sp in zip(steps, self.training_history['samples_per_second']) if not pd.isna(sp)]
            if valid_speeds:
                s_speed, speeds = zip(*valid_speeds)
                self.axes[1, 0].plot(s_speed, speeds, 'orange', linewidth=2)
        self.axes[1, 0].set_title('⚡ 学習速度')
        self.axes[1, 0].set_xlabel('ステップ')
        self.axes[1, 0].set_ylabel('サンプル/秒')
        self.axes[1, 0].grid(True, alpha=0.3)
        
        # 学習率のプロット
        if any(not pd.isna(x) for x in self.training_history['learning_rate']):
            valid_lrs = [(s, lr) for s, lr in zip(steps, self.training_history['learning_rate']) if not pd.isna(lr)]
            if valid_lrs:
                s_lr, lrs = zip(*valid_lrs)
                self.axes[1, 1].plot(s_lr, lrs, 'purple', linewidth=2)
        self.axes[1, 1].set_title('📊 学習率')
        self.axes[1, 1].set_xlabel('ステップ')
        self.axes[1, 1].set_ylabel('学習率')
        self.axes[1, 1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    
    def generate_training_report(self):
        """学習レポートを生成"""
        if not self.training_history['step']:
            print("⚠️ レポート生成用データがありません")
            return
        
        print("\\n" + "="*60)
        print("📋 SFT学習レポート")
        print("="*60)
        
        # 基本統計
        total_steps = len(self.training_history['step'])
        final_loss = [l for l in self.training_history['loss'] if not pd.isna(l)]
        final_loss = final_loss[-1] if final_loss else "N/A"
        
        max_gpu_memory = max(self.training_history['gpu_memory'])
        avg_speed = [s for s in self.training_history['samples_per_second'] if not pd.isna(s)]
        avg_speed = sum(avg_speed) / len(avg_speed) if avg_speed else "N/A"
        
        print(f"📊 基本統計:")
        print(f"  ├─ 総ステップ数: {total_steps}")
        print(f"  ├─ 最終損失: {final_loss}")
        print(f"  ├─ 最大GPU使用量: {max_gpu_memory:.1f} GB")
        print(f"  └─ 平均学習速度: {avg_speed if avg_speed != 'N/A' else 'N/A'} サンプル/秒")
        
        # 時間統計
        if len(self.training_history['timestamp']) >= 2:
            start_time = self.training_history['timestamp'][0]
            end_time = self.training_history['timestamp'][-1]
            duration = (end_time - start_time).total_seconds()
            
            print(f"\\n⏱️  時間統計:")
            print(f"  ├─ 開始時刻: {start_time.strftime('%H:%M:%S')}")
            print(f"  ├─ 終了時刻: {end_time.strftime('%H:%M:%S')}")
            print(f"  └─ 学習時間: {duration/60:.1f} 分")
        
        print("="*60)

# モニターの初期化
training_monitor = TrainingMonitor()

# デモ用のプログレス表示
def demo_training_progress():
    """デモ用の学習進捗表示"""
    print("🔄 デモ: 学習進捗モニタリング")
    print("（実際の学習時にはリアルタイムで更新されます）")
    
    # サンプルデータでデモ
    import numpy as np
    
    steps = range(0, 100, 10)
    losses = [2.5 * np.exp(-s/50) + 0.5 + np.random.normal(0, 0.1) for s in steps]
    
    for i, (step, loss) in enumerate(zip(steps, losses)):
        training_monitor.update_metrics(
            step=step,
            loss=loss,
            learning_rate=2e-5 * (1 - step/100),
            samples_per_second=15 + np.random.normal(0, 2)
        )
    
    # グラフを表示
    fig = training_monitor.start_monitoring()
    training_monitor.plot_current_progress()
    training_monitor.generate_training_report()

print("📊 学習進捗モニタリング準備完了")
print("\\n使用方法:")
print("1. demo_training_progress()  # デモ表示")
print("2. training_monitor.start_monitoring()  # リアルタイム監視開始")
print("3. 学習中に training_monitor.update_metrics() で更新")
print("4. training_monitor.generate_training_report()  # 最終レポート")

In [ ]:
# 📊 ノートブック内完結型進捗モニタリング
from IPython.display import clear_output

class NotebookTrainingMonitor:
    """ノートブック内で完結する学習進捗監視クラス"""
    
    def __init__(self):
        self.training_history = {
            'step': [],
            'loss': [],
            'eval_loss': [],
            'learning_rate': [],
            'timestamp': [],
            'gpu_memory': [],
            'samples_per_second': []
        }
        self.start_time = None
        
    def start_monitoring(self):
        """モニタリングを開始"""
        self.start_time = datetime.now()
        print("🔄 学習進捗モニタリング開始")
        print(f"開始時刻: {self.start_time.strftime('%Y-%m-%d %H:%M:%S')}")
        
    def update_metrics(self, step, loss=None, eval_loss=None, learning_rate=None, samples_per_second=None):
        """メトリクスを更新してリアルタイム表示"""
        self.training_history['step'].append(step)
        self.training_history['timestamp'].append(datetime.now())
        
        # GPU メモリ使用量を取得
        if torch.cuda.is_available():
            gpu_memory = torch.cuda.memory_allocated() / 1e9
            self.training_history['gpu_memory'].append(gpu_memory)
        else:
            self.training_history['gpu_memory'].append(0)
        
        # その他のメトリクス
        self.training_history['loss'].append(loss if loss is not None else np.nan)
        self.training_history['eval_loss'].append(eval_loss if eval_loss is not None else np.nan)
        self.training_history['learning_rate'].append(learning_rate if learning_rate is not None else np.nan)
        self.training_history['samples_per_second'].append(samples_per_second if samples_per_second is not None else np.nan)
        
        # リアルタイム表示更新（5ステップごと）
        if step % 5 == 0:
            self._update_display()
    
    def _update_display(self):
        """進捗表示を更新"""
        if len(self.training_history['step']) == 0:
            return
        
        current_time = datetime.now()
        elapsed = (current_time - self.start_time).total_seconds() if self.start_time else 0
        
        # 最新の値を取得
        latest_step = self.training_history['step'][-1]
        latest_loss = self.training_history['loss'][-1]
        latest_gpu = self.training_history['gpu_memory'][-1]
        latest_speed = self.training_history['samples_per_second'][-1]
        
        # コンソール表示更新
        print(f"\\r📊 ステップ {latest_step} | ", end="")
        if not np.isnan(latest_loss):
            print(f"損失: {latest_loss:.4f} | ", end="")
        if latest_gpu > 0:
            print(f"GPU: {latest_gpu:.1f}GB | ", end="")
        if not np.isnan(latest_speed):
            print(f"速度: {latest_speed:.1f} samples/s | ", end="")
        print(f"経過: {elapsed/60:.1f}分", end="", flush=True)
    
    def plot_training_progress(self):
        """学習進捗をプロット（ノートブック内表示）"""
        if not self.training_history['step']:
            print("⚠️ プロット用データがありません")
            return
        
        # 2x2のサブプロット
        fig, axes = plt.subplots(2, 2, figsize=(14, 10))
        fig.suptitle('🔄 LLaDA SFT学習進捗', fontsize=16, fontweight='bold')
        
        steps = self.training_history['step']
        
        # 損失プロット
        valid_losses = [(s, l) for s, l in zip(steps, self.training_history['loss']) if not np.isnan(l)]
        if valid_losses:
            s_loss, losses = zip(*valid_losses)
            axes[0, 0].plot(s_loss, losses, 'b-', linewidth=2, label='学習損失')
        
        valid_eval_losses = [(s, l) for s, l in zip(steps, self.training_history['eval_loss']) if not np.isnan(l)]
        if valid_eval_losses:
            s_eval, eval_losses = zip(*valid_eval_losses)
            axes[0, 0].plot(s_eval, eval_losses, 'r--', linewidth=2, label='検証損失')
        
        axes[0, 0].set_title('📉 学習損失')
        axes[0, 0].set_xlabel('ステップ')
        axes[0, 0].set_ylabel('損失')
        axes[0, 0].legend()
        axes[0, 0].grid(True, alpha=0.3)
        
        # GPU メモリプロット
        axes[0, 1].plot(steps, self.training_history['gpu_memory'], 'g-', linewidth=2)
        axes[0, 1].set_title('🖥️ GPU メモリ使用量')
        axes[0, 1].set_xlabel('ステップ')
        axes[0, 1].set_ylabel('メモリ (GB)')
        axes[0, 1].grid(True, alpha=0.3)
        
        # 学習速度プロット
        valid_speeds = [(s, sp) for s, sp in zip(steps, self.training_history['samples_per_second']) if not np.isnan(sp)]
        if valid_speeds:
            s_speed, speeds = zip(*valid_speeds)
            axes[1, 0].plot(s_speed, speeds, 'orange', linewidth=2)
        axes[1, 0].set_title('⚡ 学習速度')
        axes[1, 0].set_xlabel('ステップ')
        axes[1, 0].set_ylabel('サンプル/秒')
        axes[1, 0].grid(True, alpha=0.3)
        
        # 学習率プロット
        valid_lrs = [(s, lr) for s, lr in zip(steps, self.training_history['learning_rate']) if not np.isnan(lr)]
        if valid_lrs:
            s_lr, lrs = zip(*valid_lrs)
            axes[1, 1].plot(s_lr, lrs, 'purple', linewidth=2)
        axes[1, 1].set_title('📊 学習率')
        axes[1, 1].set_xlabel('ステップ')
        axes[1, 1].set_ylabel('学習率')
        axes[1, 1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    
    def generate_final_report(self):
        """最終レポートを生成（ノートブック内表示）"""
        if not self.training_history['step']:
            print("⚠️ レポート生成用データがありません")
            return
        
        print("\\n" + "="*60)
        print("📋 SFT学習最終レポート")
        print("="*60)
        
        # 基本統計
        total_steps = len(self.training_history['step'])
        final_loss = [l for l in self.training_history['loss'] if not np.isnan(l)]
        final_loss = final_loss[-1] if final_loss else "N/A"
        
        max_gpu_memory = max(self.training_history['gpu_memory']) if self.training_history['gpu_memory'] else 0
        avg_speed = [s for s in self.training_history['samples_per_second'] if not np.isnan(s)]
        avg_speed = sum(avg_speed) / len(avg_speed) if avg_speed else "N/A"
        
        print(f"📊 基本統計:")
        print(f"  ├─ 総ステップ数: {total_steps}")
        print(f"  ├─ 最終損失: {final_loss}")
        print(f"  ├─ 最大GPU使用量: {max_gpu_memory:.1f} GB")
        print(f"  └─ 平均学習速度: {avg_speed if avg_speed != 'N/A' else 'N/A'} サンプル/秒")
        
        # 時間統計
        if len(self.training_history['timestamp']) >= 2:
            start_time = self.training_history['timestamp'][0]
            end_time = self.training_history['timestamp'][-1]
            duration = (end_time - start_time).total_seconds()
            
            print(f"\\n⏱️  時間統計:")
            print(f"  ├─ 開始時刻: {start_time.strftime('%H:%M:%S')}")
            print(f"  ├─ 終了時刻: {end_time.strftime('%H:%M:%S')}")
            print(f"  └─ 学習時間: {duration/60:.1f} 分")
        
        # 設定サマリー
        print(f"\\n⚙️  設定サマリー:")
        print(f"  ├─ 設定: {CONFIG_NAME}")
        print(f"  ├─ データサイズ: {config.dataset_size:,}")
        print(f"  ├─ バッチサイズ: {config.batch_size}")
        print(f"  ├─ エポック数: {config.num_epochs}")
        print(f"  └─ 学習率: {config.learning_rate}")
        
        print("="*60)
        print("✅ レポート生成完了")

# モニターの初期化
notebook_monitor = NotebookTrainingMonitor()

# デモ用プログレス表示
def demo_notebook_monitoring():
    """ノートブック内監視のデモ"""
    print("🔄 ノートブック内監視デモ開始")
    
    # サンプルデータでデモ
    steps = range(0, 50, 5)
    losses = [2.5 * np.exp(-s/25) + 0.5 + np.random.normal(0, 0.1) for s in steps]
    
    notebook_monitor.start_monitoring()
    
    for i, (step, loss) in enumerate(zip(steps, losses)):
        notebook_monitor.update_metrics(
            step=step,
            loss=loss,
            learning_rate=2e-5 * (1 - step/50),
            samples_per_second=15 + np.random.normal(0, 2)
        )
        time.sleep(0.1)  # デモ用遅延
    
    print("\\n\\n📊 グラフ表示:")
    notebook_monitor.plot_training_progress()
    notebook_monitor.generate_final_report()

print("📊 ノートブック内監視システム準備完了")
print("\\n使用方法:")
print("1. demo_notebook_monitoring()  # デモ表示")
print("2. notebook_monitor.start_monitoring()  # 実際の監視開始")
print("3. 学習中に自動更新")
print("4. notebook_monitor.generate_final_report()  # 最終レポート")
print("=" * 50)
print("🎯 次のステップ: セル11を実行して実行インターフェースを準備")
print("=" * 50)

# 📚 FAQ・ヘルプセクション

## ❓ よくある質問

### 🚀 実行関連

**Q: 初回実行で何をすればいいですか？**
A: 以下を順番に実行：
1. セル1-8を上から順番に実行
2. `result = notebook_runner.complete_auto_training()` を実行

**Q: どの設定を選べばいいですか？**
A: 
- 初心者・検証用: `validation` (1K サンプル, 約5分)
- 中級者・バランス: `medium` (10K サンプル, 約30分)  
- 上級者・本格学習: `production` (50K サンプル, 約2時間)

**Q: 学習が途中で止まりました**
A: 以下を試してください：
```python
# GPU メモリクリア
torch.cuda.empty_cache()
gc.collect()

# より小さな設定で再実行
result = quick_training()
```

### 💾 メモリ関連

**Q: GPU メモリ不足エラーが出ます**
A: 以下の順番で対処：
1. `torch.cuda.empty_cache()` 実行
2. より小さなバッチサイズ: `config.batch_size = 1`
3. validation設定を使用: `quick_training()`
4. カスタム設定: `custom_quick_training(dataset_size=500, batch_size=1)`

**Q: どのくらいのGPUメモリが必要ですか？**
A:
- validation: 8GB以上推奨
- medium: 12GB以上推奨  
- production: 16GB以上推奨

### 📊 結果関連

**Q: 学習結果はどこに保存されますか？**
A: `./llada_sft_output/` ディレクトリに自動保存されます。

**Q: 学習の進捗はどこで確認できますか？**
A: ノートブック内に自動表示されます。追加で `notebook_monitor.plot_training_progress()` で詳細グラフ表示可能。

**Q: 生成品質を確認したいです**
A: 学習完了後に自動評価が実行されます。手動実行は：
```python
evaluate_sft_model()
```

### 🔧 トラブルシューティング

**Q: データセットが読み込めません**
A: 自動でサンプルデータに切り替わります。問題ありません。

**Q: 学習損失が下がりません**
A: 以下を確認：
1. 学習率が適切か（デフォルト: 2e-5）
2. データ量が十分か（最低1000サンプル推奨）
3. エポック数を増やす

**Q: 生成結果が期待と違います**
A: SFT後の追加調整方法：
1. より多くのエポックで学習
2. 異なる温度設定で生成テスト
3. より大きなデータセットで再学習

## 🛠️ カスタマイズ例

### パラメータ調整
```python
# バッチサイズ調整
config.batch_size = 1  # メモリ不足時

# 学習率調整  
config.learning_rate = 1e-5  # より保守的

# データサイズ調整
config.dataset_size = 5000  # 中間サイズ
```

### 詳細設定例
```python
# 高速テスト用
result = custom_quick_training(
    dataset_size=100, 
    batch_size=1, 
    epochs=1
)

# バランス型
result = custom_quick_training(
    dataset_size=2000,
    batch_size=2, 
    epochs=2
)
```

## 🆘 緊急時の対処法

### 完全リセット手順
```python
# 1. GPU メモリクリア
torch.cuda.empty_cache()
gc.collect()

# 2. カーネル再起動（Runtime > Restart Runtime）

# 3. セル1から再実行

# 4. 最小設定で実行
result = custom_quick_training(dataset_size=100, batch_size=1, epochs=1)
```

### エラー報告
エラーが解決しない場合は、以下の情報と共にご報告ください：
- 実行したコマンド
- エラーメッセージ全文
- GPU情報（`nvidia-smi`の結果）
- 使用した設定（validation/medium/production）

---

## 📞 追加サポート

**実行成功のチェックリスト:**
- ✅ セル1-8を順番に実行完了
- ✅ エラーメッセージなし
- ✅ GPU情報表示済み
- ✅ `result = notebook_runner.complete_auto_training()` 実行

**学習成功の確認:**
- ✅ 「学習完了」メッセージ表示
- ✅ 最終損失が表示される
- ✅ モデル保存完了メッセージ
- ✅ 自動評価結果表示

すべて確認できれば学習成功です！🎉

In [ ]:
# 🎮 完全自動実行システム（ノートブック内完結）

class NotebookSFTRunner:
    """ノートブック内完結型のSFT学習実行システム"""
    
    def __init__(self):
        self.is_training = False
        self.training_result = None
        
    def complete_auto_training(self, config_name="validation", show_progress=True):
        """
        完全自動SFT学習実行（ノートブック内完結）
        
        Args:
            config_name: 使用する設定 ("validation", "medium", "production")
            show_progress: 進捗表示を行うかどうか
        """
        
        print("🎬 LLaDA SFT学習 完全自動実行を開始します")
        print("="*60)
        print("🔄 ノートブック内完結システム:")
        print("  ✅ 外部ツール不要")
        print("  ✅ WandB不使用")
        print("  ✅ 進捗はノートブック内表示")
        print("  ✅ 結果はセル出力に保存")
        print("="*60)
        
        # 1. 環境チェック
        print("\\n🔍 1. 環境チェック...")
        gpu_available = torch.cuda.is_available()
        if gpu_available:
            gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
            print(f"  ✅ GPU: {torch.cuda.get_device_name(0)}")
            print(f"  ✅ メモリ: {gpu_memory:.1f} GB")
            
            # 自動設定選択
            if gpu_memory >= 15 and config_name == "validation":
                recommended_config = "medium"
                print(f"  💡 十分なメモリがあります。{recommended_config}設定を推奨")
            else:
                recommended_config = config_name
        else:
            recommended_config = "validation"
            print("  ⚠️  GPU未検出 - CPU学習（validation設定固定）")
        
        # 2. 設定適用
        print(f"\\n⚙️  2. 設定適用: {recommended_config}")
        global config, CONFIG_NAME
        CONFIG_NAME = recommended_config
        config = select_config(recommended_config)
        
        # 3. メモリ最適化
        print("\\n🧹 3. メモリ最適化...")
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            gc.collect()
            print(f"  ✅ GPU メモリクリア完了")
        
        # 4. 進捗監視開始
        if show_progress:
            print("\\n📊 4. 進捗監視開始...")
            notebook_monitor.start_monitoring()
        
        # 5. 学習実行
        print("\\n🚀 5. 学習開始...")
        print("⏰ 3秒後に開始...")
        time.sleep(3)
        
        try:
            # 実際の学習実行
            training_result = self._execute_training(show_progress)
            
            # 6. 結果表示
            print("\\n🎉 学習完了!")
            self._display_results(training_result, show_progress)
            
            # 7. 自動評価
            print("\\n🔍 7. 自動評価実行...")
            self._run_auto_evaluation()
            
            # 8. 最終レポート
            if show_progress:
                print("\\n📄 8. 最終レポート生成...")
                notebook_monitor.plot_training_progress()
                notebook_monitor.generate_final_report()
            
            # 成功メッセージ
            print("\\n" + "🎊" * 20)
            print("🏆 LLaDA SFT学習が正常に完了しました!")
            print("🎊" * 20)
            
            return training_result
            
        except Exception as e:
            print(f"\\n❌ 学習エラー: {e}")
            self._display_troubleshooting()
            raise
    
    def _execute_training(self, show_progress=True):
        """実際の学習を実行"""
        
        # 学習前チェック
        print(f"📋 学習設定確認:")
        print(f"  - データセット: {len(train_dataset)} サンプル")
        print(f"  - バッチサイズ: {config.batch_size}")
        print(f"  - エポック数: {config.num_epochs}")
        print(f"  - 推定時間: {self._estimate_time()} 分")
        
        # 学習前評価
        if val_dataset and len(val_dataset) > 0:
            print("\\n📊 学習前評価...")
            try:
                initial_eval = trainer.evaluate()
                print(f"  初期損失: {initial_eval['eval_loss']:.4f}")
            except Exception as e:
                print(f"  初期評価スキップ: {e}")
        
        # メイン学習
        print("\\n🎯 メイン学習実行中...")
        start_time = time.time()
        
        # Trainer自体の学習実行
        training_result = trainer.train()
        
        training_duration = time.time() - start_time
        print(f"\\n⏱️  学習時間: {training_duration/60:.1f} 分")
        
        return training_result
    
    def _display_results(self, training_result, show_progress):
        """結果を表示"""
        print("\\n📊 学習結果:")
        print(f"  ✅ 最終損失: {training_result.training_loss:.4f}")
        print(f"  ✅ 学習時間: {training_result.metrics['train_runtime']:.1f} 秒")
        print(f"  ✅ サンプル/秒: {training_result.metrics['train_samples_per_second']:.2f}")
        
        # モデル保存
        print("\\n💾 モデル保存中...")
        try:
            trainer.save_model()
            tokenizer.save_pretrained(config.output_dir)
            print(f"  ✅ 保存完了: {config.output_dir}")
        except Exception as e:
            print(f"  ⚠️  保存エラー: {e}")
    
    def _run_auto_evaluation(self):
        """自動評価を実行"""
        model.eval()
        
        test_questions = [
            "日本の首都はどこですか？",
            "機械学習とは何ですか？",
            "Pythonでリストを逆順にする方法を教えてください。"
        ]
        
        print(f"\\n🧪 {len(test_questions)} 個の質問で評価中...")
        
        for i, question in enumerate(test_questions, 1):
            print(f"\\n--- 評価 {i} ---")
            print(f"質問: {question}")
            
            try:
                # プロンプト準備
                formatted_text, _ = processor.format_for_sft(question, "")
                prompt_only = formatted_text.replace(processor.special_tokens['eos'], "")
                
                # 生成実行
                inputs = tokenizer(prompt_only, return_tensors="pt", max_length=256, truncation=True).to(device)
                
                with torch.no_grad():
                    outputs = model.generate(
                        **inputs,
                        max_new_tokens=80,
                        do_sample=True,
                        temperature=0.7,
                        top_p=0.9,
                        pad_token_id=tokenizer.pad_token_id
                    )
                    
                    generated_text = tokenizer.decode(
                        outputs[0][inputs.input_ids.shape[1]:], 
                        skip_special_tokens=True
                    )
                    
                    print(f"回答: {generated_text.strip()}")
                    
            except Exception as e:
                print(f"評価エラー: {e}")
        
        print("\\n✅ 自動評価完了")
    
    def _estimate_time(self):
        """学習時間を推定"""
        base_time = 0.05  # 秒/サンプル
        total_samples = config.dataset_size * config.num_epochs
        estimated_seconds = total_samples * base_time / config.batch_size
        return max(1, int(estimated_seconds / 60))
    
    def _display_troubleshooting(self):
        """トラブルシューティング表示"""
        print("\\n" + "🔧" * 20)
        print("トラブルシューティングガイド")
        print("🔧" * 20)
        print("💡 解決策:")
        print("  1. より小さな設定を試す: complete_auto_training('validation')")
        print("  2. GPU メモリクリア: torch.cuda.empty_cache()")
        print("  3. バッチサイズ削減: config.batch_size = 1")
        print("  4. セル再起動後に再実行")
        print("🔧" * 20)

# 統合実行システム
notebook_runner = NotebookSFTRunner()

# 簡単実行関数群
def quick_training():
    """最速実行（validation設定）"""
    print("⚡ クイック学習開始...")
    return notebook_runner.complete_auto_training("validation")

def medium_training():
    """中規模学習（medium設定）"""
    print("💪 中規模学習開始...")
    return notebook_runner.complete_auto_training("medium")

def production_training():
    """本格学習（production設定）"""
    print("🔥 本格学習開始...")
    return notebook_runner.complete_auto_training("production")

def custom_quick_training(dataset_size=500, batch_size=1, epochs=1):
    """カスタム設定での高速学習"""
    print(f"🎛️ カスタム学習: {dataset_size}サンプル, バッチ{batch_size}, {epochs}エポック")
    
    # 設定カスタマイズ
    global config
    config.dataset_size = dataset_size
    config.batch_size = batch_size
    config.num_epochs = epochs
    
    return notebook_runner.complete_auto_training("validation")

# メインインターフェース
print("🎯 ノートブック内完結SFT学習システム準備完了")
print("\\n" + "🚀" * 25)
print("即座に実行可能なコマンド:")
print("🚀" * 25)
print()
print("🥇 完全自動（初回推奨）:")
print("   result = notebook_runner.complete_auto_training()")
print()
print("⚡ クイック実行:")
print("   result = quick_training()")
print()
print("💪 中規模実行:")
print("   result = medium_training()")
print()
print("🔥 本格実行:")
print("   result = production_training()")
print()
print("🎛️ カスタム実行:")
print("   result = custom_quick_training(dataset_size=1000, batch_size=2, epochs=1)")
print()
print("🚀" * 25)
print("✨ 特徴:")
print("  ✅ ノートブック内完結（外部ツール不要）")
print("  ✅ 自動エラー回復")
print("  ✅ リアルタイム進捗表示")
print("  ✅ 自動評価・レポート生成")
print("🚀" * 25)